In [7]:
import pandas as pd
import folium
from IPython.display import display

# 1. 데이터 불러오기 (다시!)
pop_df = pd.read_excel("/home/gmlwls5168/code/gongmo/src/data/gwangjin_population.csv", engine="openpyxl")

# (차량등 데이터도 혹시 몰라 같이 변경)
light_df = pd.read_csv("/home/gmlwls5168/code/gongmo/src/data/gwangjin_stat.csv", encoding="utf-8-sig")



# 2. 노인 인구 (65세 이상)만 추출해서 동별 합산
elderly_cols = ["65~69세", "70~74세", "75~79세", "80~84세", "85세 이상"]
pop_df["노인인구수"] = pop_df[elderly_cols].sum(axis=1)

# 동별 필요한 컬럼만 정리
pop_df = pop_df[["동", "노인인구수"]]

# 3. 차량등 데이터 정제 (~ 앞부분만 구간 정리)
light_df["구간_정제"] = light_df["구간"].str.split("~").str[0]
light_df["도로명"] = light_df["구간_정제"].str.split().str[0]

# 도로명 → 동 매핑 테이블
gugan_to_dong = {
    "천호대로": "중곡동",
    "동일로": "화양동",
    "올림픽대교북단": "광장동",
    "영동대교 북단": "자양4동",
    "답십리로": "중곡1동",
    "광장로": "광장동",
    "군자로": "군자동",
    "면목로": "군자동",
    "아차산로": "화양동",
    "아차산로36길": "자양동",
    "광나루로30길": "구의2동",
    "광나루로56길": "구의3동",
    "강변역로": "구의1동",
    "구의로": "구의1동",
    "구의강변로": "구의3동",
    "능동로": "능동",
    "자양로": "자양2동",
    "자양로13길": "자양3동",
    "자양번영로": "자양4동",
    "뚝섬로34길": "자양4동",
    "뚝섬로52길": "자양4동",
    "뚝섬로64길": "자양4동",
    "로데오거리": "자양1동",
    "영화사로": "중곡3동",
    "용마산로": "중곡1동",
    "긴고랑로": "중곡2동",
    "워커힐로": "광장동",
    "구천면로": "광장동",
    "대학문화의거리": "화양동",
    "천호대로140길": "광장동"
}

light_df["동"] = light_df["도로명"].map(gugan_to_dong)

# 동별 차량등 개수 합산
car_light_df = light_df.groupby("동")["차도등 개수"].sum().reset_index()
car_light_df.columns = ["동", "차량등합계"]

# 4. 노인인구 + 차량등 데이터 병합
merged_df = pd.merge(pop_df, car_light_df, on="동", how="inner")

# 5. 위험지수 계산
merged_df["위험지수"] = merged_df["노인인구수"] / merged_df["차량등합계"]

# 결과 출력
print(merged_df)

# 6. folium 지도용 좌표 매핑
dong_coords = {
    "화양동": (37.5421, 127.0722),
    "군자동": (37.5578, 127.0790),
    "중곡1동": (37.5654, 127.0806),
    "중곡2동": (37.5624, 127.0784),
    "중곡3동": (37.5628, 127.0873),
    "중곡4동": (37.5646, 127.0902),
    "능동": (37.5491, 127.0796),
    "구의1동": (37.5388, 127.0922),
    "구의2동": (37.5350, 127.0892),
    "구의3동": (37.5381, 127.0854),
    "광장동": (37.5452, 127.1040),
    "자양1동": (37.5313, 127.0826),
    "자양2동": (37.5338, 127.0745),
    "자양3동": (37.5384, 127.0689),
    "자양4동": (37.5409, 127.0582)
}

merged_df["위도"] = merged_df["동"].map(lambda x: dong_coords.get(x, (None, None))[0])
merged_df["경도"] = merged_df["동"].map(lambda x: dong_coords.get(x, (None, None))[1])

# 7. folium 지도 표시
risk_map = folium.Map(location=[merged_df["위도"].mean(), merged_df["경도"].mean()], zoom_start=13)

for _, row in merged_df.iterrows():
    folium.CircleMarker(
        location=[row["위도"], row["경도"]],
        radius=row["위험지수"] * 0.2,  # 위험지수 비례 반지름 (조정 가능)
        color="crimson",
        fill=True,
        fill_opacity=0.7,
        popup=(
            f"동: {row['동']}<br>"
            f"노인인구수: {row['노인인구수']}명<br>"
            f"차량등합계: {row['차량등합계']}개<br>"
            f"위험지수: {row['위험지수']:.2f}"
        )
    ).add_to(risk_map)

# 지도 출력
display(risk_map)


ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.